In [2]:
"""
Author: Annam.ai IIT Ropar
Team Members: Aman Sagar
Leaderboard Rank: 16

"""


from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

dataset_path = '/content/drive/My Drive/soil_classification-2025'
os.listdir(dataset_path)


#Data Preprocessing: Resize images, normalize pixel values, and apply data augmentation as needed.

In [ ]:
!pip install -q pandas scikit-learn torchvision

import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split


In [ ]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [ ]:
import torch
print("PyTorch version:", torch.__version__)


PyTorch version: 2.7.0+cu118


In [ ]:

ROOT_DIR = "/content/drive/MyDrive/soil_classification-2025"
TRAIN_CSV = os.path.join(ROOT_DIR, "train_labels.csv")
TEST_CSV = os.path.join(ROOT_DIR, "test_ids.csv")
TRAIN_DIR = os.path.join(ROOT_DIR, "train")
TEST_DIR = os.path.join(ROOT_DIR, "test")


In [ ]:
label_map = {
    "Alluvial soil": 0,
    "Black Soil": 1,
    "Clay soil": 2,
    "Red soil": 3
}

df = pd.read_csv(TRAIN_CSV)
df['label'] = df['soil_type'].map(label_map)
# df_train, df_val = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
df_train = df  # use the full training set


In [ ]:
# IMAGE_SIZE = 224

# train_transforms = transforms.Compose([
#     transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(10),
#     transforms.ColorJitter(brightness=0.2, contrast=0.2),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet mean
#                          [0.229, 0.224, 0.225])  # ImageNet std
# ])

# val_transforms = transforms.Compose([
#     transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406],
#                          [0.229, 0.224, 0.225])
# ])

train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.75, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),

    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
class SoilDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, test=False):
        self.df = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.test = test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.iloc[idx]['image_id']
        img_path = os.path.join(self.img_dir, img_id)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        if self.test:
            return image, img_id
        else:
            label = self.df.iloc[idx]['label']
            return image, label


Index(['image_id', 'soil_type', 'label'], dtype='object')


In [ ]:
BATCH_SIZE = 32

train_dataset = SoilDataset(df_train, TRAIN_DIR, transform=train_transforms)
# val_dataset = SoilDataset(df_val, TRAIN_DIR, transform=val_transforms)
val_dataset = SoilDataset(df_val, TRAIN_DIR, transform=val_transforms)


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


In [ ]:
test_df = pd.read_csv(TEST_CSV)
test_dataset = SoilDataset(test_df, TEST_DIR, transform=val_transforms, test=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


# Step 2: Build Model, Training & Validation Loop


In [ ]:
import torch
import torch.nn as nn
from torchvision import models
import torch.optim as optim
from tqdm import tqdm  # nice progress bars

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# Load pretrained ResNet50 and modify for 4 classes
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = True
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 4)
model = model.to(device)


Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


#Step 3: Define Loss and Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', patience=2, factor=0.5
)


#Step 4: Training and Validation Functions

In [ ]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct_preds = 0

    for inputs, labels in tqdm(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_preds += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct_preds.double() / len(dataloader.dataset)
    return epoch_loss, epoch_acc.item()


def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_preds = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct_preds.double() / len(dataloader.dataset)
    return epoch_loss, epoch_acc.item()


#Step 5: Run Training Loop

In [ ]:
# EPOCHS = 10

# for epoch in range(EPOCHS):
#     train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
#     val_loss, val_acc = validate(model, val_loader, criterion, device)
#     scheduler.step(val_loss)


#     print(f"Epoch {epoch+1}/{EPOCHS}")
#     print(f"Train loss: {train_loss:.4f}, Train accuracy: {train_acc:.4f}")
#     print(f"Val loss: {val_loss:.4f}, Val accuracy: {val_acc:.4f}")



!pip install -q timm

import timm
from torch.optim.lr_scheduler import ReduceLROnPlateau

model = timm.create_model('efficientnet_b3a', pretrained=True, num_classes=4)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', patience=2, factor=0.5
)

best_val_acc = 0
patience, trigger = 3, 0

EPOCHS = 20
for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    scheduler.step(val_loss)

    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"Train loss: {train_loss:.4f}, acc: {train_acc:.4f}")
    print(f"Val   loss: {val_loss:.4f}, acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pth")
        trigger = 0
    else:
        trigger += 1
        if trigger >= patience:
            print("Early stopping.")
            break


/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name efficientnet_b3a to current efficientnet_b3.
  model = create_fn(
100%|██████████| 39/39 [00:21<00:00,  1.84it/s]


Epoch 1/20
Train loss: 0.7872, acc: 0.8093
Val   loss: 0.1390, acc: 0.9469


100%|██████████| 39/39 [00:18<00:00,  2.13it/s]


Epoch 2/20
Train loss: 0.2129, acc: 0.9288
Val   loss: 0.1533, acc: 0.9510


100%|██████████| 39/39 [00:19<00:00,  2.00it/s]


...

Epoch 5/20
Train loss: 0.1270, acc: 0.9632
Val   loss: 0.1115, acc: 0.9592


100%|██████████| 39/39 [00:19<00:00,  1.99it/s]


Epoch 6/20
Train loss: 0.1019, acc: 0.9673
Val   loss: 0.0727, acc: 0.9714


100%|██████████| 39/39 [00:18<00:00,  2.16it/s]


Epoch 7/20
Train loss: 0.0635, acc: 0.9779
Val   loss: 0.0495, acc: 0.9837
Early stopping.


#Step 6: Predict on Test Set and Prepare Submission


In [ ]:
# model.eval()
# all_preds = []
# all_img_ids = []

# with torch.no_grad():
#     for inputs, img_ids in test_loader:
#         inputs = inputs.to(device)
#         outputs = model(inputs)
#         _, preds = torch.max(outputs, 1)
#         all_preds.extend(preds.cpu().numpy())
#         all_img_ids.extend(img_ids)


from torchvision.transforms import functional as TF
import numpy as np

model.eval()
all_preds = []
all_img_ids = []

tta_transforms = [
    lambda x: x,
    lambda x: TF.hflip(x),
    lambda x: TF.vflip(x),
    lambda x: TF.rotate(x, 15),
]

with torch.no_grad():
    for inputs, img_ids in test_loader:
        batch_preds = []
        for tform in tta_transforms:
            augmented = torch.stack([tform(img.cpu()) for img in inputs])
            augmented = augmented.to(device)
            outputs = model(augmented)
            preds = torch.softmax(outputs, dim=1)
            batch_preds.append(preds.cpu().numpy())
        mean_preds = np.mean(batch_preds, axis=0)
        final_preds = np.argmax(mean_preds, axis=1)
        all_preds.extend(final_preds)
        all_img_ids.extend(img_ids)

# Reverse label_map to get soil_type string from label index
inv_label_map = {v: k for k, v in label_map.items()}
predicted_soil_types = [inv_label_map[p] for p in all_preds]

import pandas as pd
submission = pd.DataFrame({
    "image_id": all_img_ids,
    "soil_type": predicted_soil_types
})

submission.to_csv("submission5.csv", index=False)
print("Submission file created: submission5.csv")





# model.load_state_dict(torch.load("best_model.pth"))
# model.eval()

# tta_transforms = [
#     transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406],
#                              [0.229, 0.224, 0.225])
#     ]),
#     transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.RandomHorizontalFlip(p=1.0),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406],
#                              [0.229, 0.224, 0.225])
#     ])
# ]

# all_preds = []
# all_img_ids = []

# for tta_transform in tta_transforms:
#     tta_dataset = SoilDataset(test_df, TEST_DIR, transform=tta_transform, test=True)
#     tta_loader = DataLoader(tta_dataset, batch_size=BATCH_SIZE, shuffle=False)

#     preds = []
#     with torch.no_grad():
#         for inputs, img_ids in tta_loader:
#             inputs = inputs.to(device)
#             outputs = model(inputs)
#             preds.append(outputs.cpu())

#     all_preds.append(torch.cat(preds))

# final_outputs = sum(all_preds) / len(all_preds)
# final_preds = torch.argmax(final_outputs, dim=1).numpy()
# img_ids = test_df['image_id'].tolist()

# # Map predictions
# inv_label_map = {v: k for k, v in label_map.items()}
# predicted_soil_types = [inv_label_map[p] for p in final_preds]

# submission = pd.DataFrame({
#     "image_id": img_ids,
#     "soil_type": predicted_soil_types
# })
# submission.to_csv("submission4.csv", index=False)
# print("Created: submission4.csv")


Submission file created: submission5.csv


In [ ]:
from google.colab import files
files.download("submission5.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>